<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js086_crystalGrowthKMC2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [25]:
#@title js086_crystalGrowthKMC2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.11.28 created,  last updated on 2024.03.05
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js086_crystalGrowthKMC2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js086_crystalGrowthKMC2D
//    Copyright(C) 2018-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2018.01.20 created, last updated on 2018.11.30
//    ver 0.0.1  2019.01.23 v1, last updated on 2021.08.15
//    ver 0.0.2  2021.11.05 v2, last updated on 2021.11.05
//    ver 0.0.3  2023.04.27 v3, last updated on 2023.09.04
//
// -------------------- kinetic Monte-Carlo algorithm KMC2D
//
//  - Bortz-Kalos-Liebowitz (BKL) algorithm
//
//    i-th transition rate in the system : ri
//    total transition rate : RN = sum(ri, {i=1,2,...,N})
//
//  procedure
//
//    (1) t = 0
//
//    (2) select i : randomly (proportional to ri/RN)
//      calculate R(i) = sum(rj, {j=1,2,...,i})
//      get a uniform random number u in [0,1]
//      find i  R(i-1) < u RN <= R(i)
//
//    (3) execute i-th transition
//
//    (4) update the time
//      get a uniform random numner u in (0,1]
//      dt = -log(u)/R
//      t = t + dt
//
//    goto (2)
//
//  transion rate k = nue0*exp(-dE/kT)
//      nue0 = kT/h , h: Plank's constant
//
//  periocic condition
//
//  faster:
//      In the calculation of the sum of the transition probabilities,
//      the sum is calculated from the change in the transition probability near the movded particle.
//
// --------------------
*/

const crystalGrowthKMC2D = (function(){ // ====================  crystalGrowthKMC2D Module  ====================

	const g_hPlank = 6.62607015e-34;			// (Js) Plank's constant
	const g_EE = 1.602176634e-19;				// (C) electron charge, energy : 1(eV) = g_EE(J)
	const g_nMax = 10000;						// array max
	const g_nxMax = 200;						// NNx max
	const g_nyMax = 200;						// NNy max

	let g_initNNa = 30*30;						// initial number of adatom
	let g_NNa = g_initNNa;						// number of adatom
	let g_maxNNa = 1800;						// g_NNa limit (= initial and added adatom)
	let g_NNx = 72;								// x-division of the field
	let g_NNy = 72;								// y-division of the field
	let g_sysTime = 0.0;						// (s) system time
	let g_qqHopping = 0.2;						// (eV) hopping energy
	let g_qqNeighbor = 0.3;						// (eV) adsorption energy to neighbor adatom
	let g_kT = 0.05;							// (eV) thermal energy to random walk
	let g_changeFlag = 1;						// if (g_changeFlag = 1) setAllTransition()
	let g_totalRate = 1.0e15;					// total transition rate
	let g_addingRate = 0.05;					// if (Math.random()<g_addingRate) add adatom

	const g_lattice = dimInt2( g_nxMax, g_nyMax ); // g_lattice[ix][iy]  0...g_NNa: adatom, -1:free space
	const g_ixAdatom = dimInt1( g_nMax );		// g_ixAdatom[i] x-position of i-th adatom
	const g_iyAdatom = dimInt1( g_nMax );		// g_iyAdatom[i] y-position of i-th adatom
	const g_kind =  dimInt1( g_nMax );			// adatom kind: 0:initial adatom 1:added adatom
	const g_transition = dim1( 4*g_nMax );		// transition rate of direction E/N/W/S adatom
	const g_transitionAdatom = dim1( g_nMax );	// transition rate of adatom
	const g_rate = dim1( 5 );					// g_rate[] = transition rate ( see setRate() )


	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme ) {
		// theme -- no use

		g_sysTime = 0.0;
		clearField();
		setCrystal(21, 21, 30, 30);
		g_initNNa = 30*30;
		g_NNa = g_initNNa;
		addAdatom(g_maxNNa);
		//g_totalRate = setAllTransition();
		g_changeFlag = 1;
	}

	function clearField() {
		const nnx=g_NNx, nny=g_NNy;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				g_lattice[i][j] = -1;
			}
		}
	}

	function setCrystal(ix, iy, nx, ny) {
		let ia = 0;
		for (let i=ix; i<ix+nx; i++) {
			for (let j=iy; j<iy+ny; j++) {
				g_lattice[i][j] = ia;
				g_ixAdatom[ia] = i; g_iyAdatom[ia] = j;
				g_kind[ia] = 0; // initial adatom
				ia += 1;
			}
		}
		g_NNa = ia;
	}

	function addAdatom(mxNNa) {
		const nnx=g_NNx,nny=g_NNy;
		if (g_NNa<mxNNa) {
			const ia = g_NNa;
			let x, y;
			do {
				x = Math.floor(nnx*Math.random());
				y = Math.floor(nny*Math.random());
			} while (g_lattice[x][y]!=-1); // g_lattice[][] >=0: particle, -1:free space
			g_lattice[x][y] = ia;
			g_ixAdatom[ia] = x;
			g_iyAdatom[ia] = y;
			g_kind[ia] = 1; // added adatom
			g_NNa += 1;
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc, qqHop, qqNbr, TEnergy, isChange ) {

		if (isChange) {
			g_qqHopping = qqHop;
			g_qqNeighbor = qqNbr;
			g_kT = TEnergy;
			g_changeFlag = 1;
		}

		for (let ii=0; ii<nCalc; ii++) {
			if (Math.random()<g_addingRate) {
				addAdatom(g_maxNNa);
				g_changeFlag = 1;
			}
			const nn = 1*Math.min(200,g_NNa/10 + 1);
			for (let i=0; i<nn; i++) {
				const deltat = kmcStep();
				g_sysTime += deltat;
			}
			correctTotalRate();
		}
	}

	function correctTotalRate() {
		const nna=g_NNa;
		let s=0.0;
		for (let i=0;i<nna;i++) {
			s += g_transitionAdatom[i];
		}
		g_totalRate = s;
	}

	function kmcStep() {
		if (g_changeFlag==1) {
			g_totalRate = setAllTransition();
			g_changeFlag = 0;
		}
		const ipp = selectTransion(g_totalRate);
		move(ipp);
		const r = 1.0-Math.random(); // r=(0,1], Math.random()= [0,1)
		return(-Math.log(r)/g_totalRate);
	}

	function setAllTransition() {
		const nna=g_NNa;
		setRate();
		let rsum = 0.0;
		for (let ia=0; ia<nna; ia++) {
			rsum += setTransition(ia);
		}
		return rsum;
	}

	function setRate() {
		g_rate[0] = transitionRate(g_qqHopping);
		g_rate[1] = transitionRate(g_qqHopping + g_qqNeighbor);
		g_rate[2] = transitionRate(g_qqHopping + 2.0*g_qqNeighbor);
		g_rate[3] = transitionRate(g_qqHopping + 3.0*g_qqNeighbor);
		g_rate[4] = transitionRate(g_qqHopping + 4.0*g_qqNeighbor);
	}

	function transitionRate(energy) {
		const nue0 = g_kT*g_EE/g_hPlank;
		return nue0*Math.exp(-energy/g_kT);
	}

	function setTransition(ia) {
		const i = g_ixAdatom[ia], j = g_iyAdatom[ia];
		const nb = numberOfNeighbor(i,j);
		g_transition[ia*4  ] = dirRate((i+1)%g_NNx, j, nb);
		g_transition[ia*4+1] = dirRate((i-1+g_NNx)%g_NNx, j, nb);
		g_transition[ia*4+2] = dirRate(i, (j+1)%g_NNy, nb);
		g_transition[ia*4+3] = dirRate(i, (j-1+g_NNy)%g_NNy, nb);
		g_transitionAdatom[ia] = g_transition[ia*4]+g_transition[ia*4+1]+g_transition[ia*4+2]+g_transition[ia*4+3];
		return g_transitionAdatom[ia];
	}

	function dirRate(ii, jj, nb) {
		let nb1, rt=0.0;
		if (g_lattice[ii][jj]>=0) {
			rt = 0.0;
		} else {
			nb1 = numberOfNeighbor(ii,jj)-1;
			rt = (nb<=nb1) ? g_rate[0] : g_rate[nb-nb1];
		}
		return rt;
	}

	function numberOfNeighbor(i,j) {
		let nb=0;
		if (g_lattice[(i+1)%g_NNx][j]>=0) nb += 1;
		if (g_lattice[(i-1+g_NNx)%g_NNx][j]>=0) nb += 1;
		if (g_lattice[i][(j+1)%g_NNy]>=0) nb += 1;
		if (g_lattice[i][(j-1+g_NNy)%g_NNy]>=0) nb += 1;
		return nb;
	}

	function selectTransion(rsum) {
		const nna=g_NNa;
		let rnd = rsum*Math.random();
		for (let ia=0; ia<nna; ia++) {
			rnd -= g_transitionAdatom[ia];
			if (rnd<0.0) {
				rnd += g_transitionAdatom[ia];
				for (let iap=0; iap<4; iap++) {
					rnd -= g_transition[ia*4+iap];
					if (rnd<0.0) return (ia*4+iap);
				}
				return(ia*4+3);
			}
		}
		return((nna-1)*4+3);
	}

	function move(ipp) {
		if (g_transition[ipp]>0.0) {
			const ia = Math.floor(ipp/4);
			const id = ipp%4;
			let i = g_ixAdatom[ia], j = g_iyAdatom[ia];
			g_lattice[i][j] = -1;
			if (id==0) i = (i+1)%g_NNx;
			if (id==1) i = (i-1+g_NNx)%g_NNx;
			if (id==2) j = (j+1)%g_NNy;
			if (id==3) j = (j-1+g_NNy)%g_NNy;
			g_ixAdatom[ia] = i; g_iyAdatom[ia] = j;
			g_lattice[i][j] = ia;

			const rs0 = rateAround(i,j);
			const rs1 = setRateAround(i,j);
			g_totalRate = g_totalRate - rs0 + rs1;
		}
	}

	function setRateAround(i0, j0) {
		const nnx=g_NNx, nny=g_NNy;
		let srate=0.0;
		for (let i=i0-2; i<=i0+2; i++) {
			const ii = (i+nnx)%nnx;
			for (let j=j0-2; j<=j0+2; j++) {
				const jj =  (j+nny)%nny;
				if (g_lattice[ii][jj]>=0) {
					srate += setTransition(g_lattice[ii][jj]);
				}
			}
		}
		return srate;
	}

	function rateAround(i0, j0) {
		const nnx=g_NNx,nny=g_NNy;
		let srate = 0.0;
		for (let i=i0-2; i<=i0+2; i++) {
			const ii = (i+nnx)%nnx;
			for (let j=j0-2; j<=j0+2; j++) {
				const jj =  (j+nny)%nny;
				if (g_lattice[ii][jj]>=0) {
					srate += g_transitionAdatom[g_lattice[ii][jj]];
				}
			}
		}
		return srate;
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme )
		evolve:			timeEvolution,			// timeEvolution( nCalc, qqHop, qqNbr, TEnergy, isChange )

		getSysParam:	function() { return [ g_NNa, g_NNx, g_NNy, g_initNNa, g_maxNNa ]; },
		getNNa:			function() { return g_NNa; },
		getNow:			function() { return [ g_sysTime, g_qqHopping, g_qqNeighbor, g_kT ]; },
		getWalker:		function(i) { return [ g_ixAdatom[i], g_iyAdatom[i] ]; },
		getLattice:		function(ix,iy) { return g_lattice[ix][iy]; },
	};

})(); // ====================  crystalGrowthKMC2D end  ====================


const js086 = (function(){ // ====================  js Module  ====================

	const theModule = crystalGrowthKMC2D;
	const xCanvasSize = 500;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');

	let v_theme = 0;			// lattice 0:40x40, 1:60x60, 2:80x80
	let v_nCalc = 1;			// 1 (calc/frame)
	let v_qqHop = 0.20;			// (eV) hopping energy
	let v_qqNbr = 0.30;			// (eV) adsorption energy to neighbor adatom
	let v_TEnergy = 0.050;		// (eV) thermal energy
	let v_isCange = true;

	let p_NNa, p_NNx, p_NNy, p_initNNa, p_maxNNa; // <-- theModule.getSysParam()
  let now_NNa; // <-- theModule.getNNa(); set NNa in draw()
  let sysTime, qqHopping, qqNeighbor, kT;
	let nowData = [];
	let kindList = [];
  let ixList = [];
  let iyList = [];

	let dispMode = 0; // no use
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	//let inStepFlag = false;

  let breakFlag = false;
  let perticleFlag = true;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d', { willReadFrequently:true } );
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme );
			[ p_NNa, p_NNx, p_NNy, p_initNNa, p_maxNNa ] = theModule.getSysParam();
			v_isCange = true;
      perticleFlag = true;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc, v_qqHop, v_qqNbr, v_TEnergy, v_isCange );
			v_isCange = false;
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc, v_qqHop, v_qqNbr, v_TEnergy, v_isCange  );
			v_isCange = false;
			//inStepFlag = true;
		}

		draw( ctx, dispMode );

    if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

  function setParticlesData() {
		kindList = [];
		ixList = [];
		iyList = [];
		nowData = [ sysTime, qqHopping, qqNeighbor, kT, p_initNNa, now_NNa  ];
    for (let i=0; i<now_NNa; i++) {
			kindList[i] = (i<p_initNNa) ? 0 : 1;
			let ix, iy;
      [ ix, iy ] = theModule.getWalker(i);
			ixList[i] = ix;
			iyList[i] = iy;
		}
	}


	// --------------------  draw  --------------------

	const colorOf = { initAdatom:"#00ff00", addedAdatom:"#ffff00" };

	function draw( ctx, dispMode ) {
		const xp = 60, yp = 40, xBoxSize = 360, yBoxSize = Math.floor(xBoxSize/p_NNx*p_NNy), sc = xBoxSize/p_NNx;
		now_NNa = theModule.getNNa();
		[ sysTime, qqHopping, qqNeighbor, kT ] = theModule.getNow()

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		// box
		ctx.strokeStyle = "#888888";
		ctx.strokeRect( xp-1, yp-1, xBoxSize+2, yBoxSize+2 );

		drawLattice( ctx, p_NNx, p_NNy, xp, yp, sc );

		ctx.fillStyle = colorOf.initAdatom;
		ctx.fillText(`inital adatom : ${p_initNNa} `, 40, 25);
		ctx.fillStyle = colorOf.addedAdatom;
		ctx.fillText(`added adatom : ${(now_NNa - p_initNNa)} `, 250, 25);

		ctx.fillStyle = "#888888";
		ctx.fillText(`periodic field = ${p_NNx} x ${p_NNy},`, 40, yCanvasSize-50);
		ctx.fillText(`number of adatom N = ${now_NNa}`, 250, yCanvasSize-50);
		ctx.fillText(`temp = ${kT} (eV) ~ ${(11602.8*kT).toFixed(1)} (K)`, 40, yCanvasSize-30);
		ctx.fillText(`time = ${(sysTime*1.0e9).toFixed(3)} (ns)`, 40, yCanvasSize-10);
	}

	function drawLattice( ctx, nnx, nny, xp, yp, sc ) {
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const fij = theModule.getLattice(i,j);
				const color = ( fij<p_initNNa ) ? colorOf.initAdatom : colorOf.addedAdatom;
				if (fij>=0) {
					drawDisc(ctx, xp+(i+0.5)*sc,yp+((nny-j-1)+0.5)*sc,0.4*sc,color);
				}
			}
		}
	}

	function drawDisc( ctx, x, y, r, color ) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setqqHop() {
		v_qqHop = Number(document.getElementById("range_qqHop").value);
		document.getElementById("text_qqHop").innerHTML = " " + v_qqHop.toFixed(2);
		v_isCange = true;
	}

	function setqqNbr() {
		v_qqNbr = Number(document.getElementById("range_qqNbr").value);
		document.getElementById("text_qqNbr").innerHTML = " " + v_qqNbr.toFixed(2);
		v_isCange = true;
	}

	function setTEnergy() {
		v_TEnergy = Number(document.getElementById("range_TEnergy").value);
		document.getElementById("text_TEnergy").innerHTML = " " + v_TEnergy.toFixed(3);
		v_isCange = true;
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setSpeed() {
		v_nCalc = 1 + document.getElementById("slct_speed").selectedIndex;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, qqHopping, qqNeighbor, kT, p_initNNa, now_NNa ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ nowData, kindList, ixList, iyList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setqqHop:		setqqHop,		// setqqHop()
		setqqNbr:		setqqNbr,		// setqqNbr()
		setTEnergy:		setTEnergy,		// setTEnergy()
		setDispMode:	setDispMode,	// setDispMode()
		setSpeed:		setSpeed,		// setSpeed()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, qqHopping, qqNeighbor, kT, p_initNNa, now_NNa ]
    pygetParticlesList: pygetParticlesList, //() :return [ nowData, kindList, ixList, iyList ]
	};

})(); // ====================  js086 module end  ====================


const js = js086;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js086] crystal growth - faster kinetic Monte-Carlo simulation 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="500" HEIGHT="480"></canvas>
<br>

<span>theme: crystal growth</span>
    <span style="margin-right: 160px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>Q hopping =</label>
<input type="range" id="range_qqHop" min="0.05" max="0.25" value="0.2" step="0.01"
style="width:300px" list="tick_qqHop" oninput="js.setqqHop()">
<datalist id="tick_qqHop"><option value="0.05"><option value="0.1">
<option value="0.15"><option value="0.20"><option value="0.25"></datalist>
<label id="text_qqHop"> 0.20</label> (eV)
<br>

<label>Q neighbor =</label>
<input type="range" id="range_qqNbr" min="0.1" max="0.5" value="0.3" step="0.01"
style="width:300px" list="tick_qqNbr" oninput="js.setqqNbr()">
<datalist id="tick_qqNbr"><option value="0.1"><option value="0.2">
<option value="0.3"><option value="0.4"><option value="0.5"></datalist>
<label id="text_qqNbr"> 0.30</label> (eV)
<br>

<label>kT(Temp) =</label>
<input type="range" id="range_TEnergy" min="0.025" max="0.2" value="0.05" step="0.005"
style="width:300px" list="tick_TEnergy" oninput="js.setTEnergy()">
<datalist id="tick_TEnergy"><option value="0.05"><option value="0.10"><option value="0.15">
<option value="0.20"><option value="0.25"></datalist>
<label id="text_TEnergy"> 0.05</label> (eV)
<br>

<label>nCalc/frame:</label>
<select id="slct_speed" onChange="js.setSpeed()">
<option selected>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option><option>7</option><option>8</option>
<option>9</option><option>10</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get system data and print
for i in range(10):
  [ sysTime, qqHopping, qqNeighbor, kT, initNNa, NNa ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2d},  time = {sysTime*1e12:>9.2f} (ps),  Q_hopping = {qqHopping:>6.2f} (eV),  Q_neighbor = {qqNeighbor:>6.2f} (eV),  n_Particle = {NNa:>4d} (init n:{initNNa:>4d})')
  time.sleep(3)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# get adatom data

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get system data and print
for i in range(10):
  [ sysTime, qqHopping, qqNeighbor, kT, initNNa, NNa ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2d},  time = {sysTime*1e12:>9.2f} (ps),  Q_hopping = {qqHopping:>6.2f} (eV),  Q_neighbor = {qqNeighbor:>6.2f} (eV),  n_Particle = {NNa:>4d} (init n:{initNNa:>4d})')
  time.sleep(3)

# get adatom data
print("")
print("-- get adatom data --")
[ nowData, kindList, ixList, iyList ] = eval_js('js.pygetParticlesList()')
[ sysTime, qqHopping, qqNeighbor, kT, initNNa, NNa ] = nowData
print(f'got time = {sysTime*1e12:>9.2f} (ps),  Q_hopping = {qqHopping:>6.2f} (eV),  Q_neighbor = {qqNeighbor:>6.2f} (eV),  n_Particle = {NNa:>4d} (init n:{initNNa:>4d})')
print(f'len(kindList) = {len(kindList)}, len(ixList) = {len(ixList)}, len(iyList) = {len(iyList)} ')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# save particles data

import numpy as np

# convert [ kindList, ixList, iyList ] to np_data
np_data = np.array([ kindList, ixList, iyList ])
print("shape of  np_data :", np_data.shape )

# save np_data
print("-- save as 'js086_data.npy'" )
np.save( 'js086_data.npy', np_data )

In [ ]:
# load data and prepare numpy data for plot

import numpy as np

colorList = np.array([ [0,1,0], [0.7,0.7,0] ])

# load np_data <-- np.array([ kindList, ixList, iyList ])
print("-- load data --")
loaded_data = np.load('js086_data.npy')

# prepare data ( numpy array ) for plot

Knd = loaded_data[0].astype(int) # kind of every adatom
X = loaded_data[1]               # x-position of every adatom
Y = loaded_data[2]               # y-position of every adatom
C = colorList[Knd]               # color of every adatom

In [ ]:
# adatom - scatter plot

import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.scatter(X, Y, c=C )
plt.xlim([0, 72])
plt.ylim([0, 72])
plt.title("adatom ( initial:green, added: yellow )")
plt.show()